# RUDI Node tools: *rudinode-read* library

This library offers tools to take advantage of the [external API](https://app.swaggerhub.com/apis/OlivierMartineau/RUDI-PRODUCER) of a RUDI Producer node (also referred as RUDI node).

File [rudi_node_reader.py](lib_rudi_meta/rudi_node_reader.py) contains a class `RudiNodeReader` that makes it easier 
to access a RUDI Producer node data.
This class fetches all the metadata once and let you access the resulting metadata.

## Initializing the `RudiNodeReader` object
This object only takes the RUDI node URL as a parameter.
You can optionaly give some identifier that will be used for every request made to the node.
> RUDI node external API does not need any identification. But you can give an indication in the request header.

In [7]:
from src.rudi_node_read.rudi_node_reader import RudiNodeReader
from src.rudi_node_read.utils.log import log_d

rudi_node_url = 'https://bacasable.fenix.rudi-univ-rennes1.fr'
rudi_node_info = RudiNodeReader(server_url=rudi_node_url, headers_user_agent='RudiNodeGet-Example01')

## Access to metadata information
The `RudiNodeReader` object lets you access and take advantage of the metadata stored on the node:
- access to the full list of metadata

In [8]:
info_tag = 'RudiNode info'

log_d(info_tag, 'metadata nb', rudi_node_info.metadata_count)
log_d(info_tag, 'metadata list', rudi_node_info.metadata_list)
log_d(info_tag, 'metadata 1', rudi_node_info.metadata_list[0])

D 2023-04-18 17:40:46 [Connector] base_url: https://bacasable.fenix.rudi-univ-rennes1.fr
D 2023-04-18 17:40:46 [RudiNodeConnector] attributes: {"scheme": "https", "host": "bacasable.fenix.rudi-univ-rennes1.fr", "path": "", "base_url": "https://bacasable.fenix.rudi-univ-rennes1.fr"}
D 2023-04-18 17:40:46 [RudiNodeConnector.request] to: https://bacasable.fenix.rudi-univ-rennes1.fr/api/admin/hash
D 2023-04-18 17:40:46 [RudiNodeConnector] Node 'bacasable.fenix.rudi-univ-rennes1.fr': connection OK
D 2023-04-18 17:40:46 [RudiNodeConnector.request] to: https://bacasable.fenix.rudi-univ-rennes1.fr/api/v1/resources?limit=1
D 2023-04-18 17:40:46 [RudiNode info] metadata nb: 1008
D 2023-04-18 17:40:46 [RudiNodeConnector.request] to: https://bacasable.fenix.rudi-univ-rennes1.fr/api/v1/resources?limit=1
D 2023-04-18 17:40:46 [RudiNodeConnector.request] to: https://bacasable.fenix.rudi-univ-rennes1.fr/api/v1/resources?limit==500&offset=0
D 2023-04-18 17:40:46 [RudiNodeConnector.request] to: https://

- access to the producers and contacts information

In [9]:
log_d(info_tag, 'list of producers', rudi_node_info.organization_list)
log_d(info_tag, 'producer names', rudi_node_info.organization_names)

log_d(info_tag, 'list of contacts', rudi_node_info.contact_list)
log_d(info_tag, 'contact names', rudi_node_info.contact_names)

D 2023-04-18 17:40:49 [RudiNode info] list of producers: [{'organization_id': 'ae643f29-73f1-468b-ab90-e3430a1a2fa2', 'organization_name': 'RUDI', 'organization_address': 'RUDI'}, {'organization_id': 'd3270a53-e6b6-4024-8ff3-3fda35ba2c8e', 'organization_name': "Ministère de l'Intérieur"}, {'organization_id': '1d6bc543-07ed-46f6-a813-958edb73d5f0', 'organization_name': 'SIB (Test)'}, {'organization_id': 'fa557d8b-0892-47aa-809b-6da59081e0aa', 'organization_name': 'Gusikowski LLC', 'organization_address': '4974 Altenwerth Wells, Brownville', 'collection_tag': 'rudi-seed'}, {'organization_id': '44f5ac9d-34d6-44d0-99a9-0496654bde5c', 'organization_name': 'Breitenberg - Legros', 'organization_address': '425 Hickle Crest, Duluth', 'collection_tag': 'rudi-seed'}]
D 2023-04-18 17:40:49 [RudiNode info] producer names: ['Breitenberg - Legros', 'Gusikowski LLC', "Ministère de l'Intérieur", 'RUDI', 'SIB (Test)']
D 2023-04-18 17:40:49 [RudiNode info] list of contacts: [{'contact_id': '70e09aca-a4cf

- access to the classification tags

In [10]:
log_d(info_tag, 'themes', rudi_node_info.themes)
log_d(info_tag, 'keywords', rudi_node_info.keywords)

D 2023-04-18 17:40:52 [RudiNode info] themes: ['children', 'citizenship', 'culture', 'economy', 'education', 'energyNetworks', 'environment', 'health', 'housing', 'location', 'publicSpace', 'society', 'townPlanning', 'transportation']
D 2023-04-18 17:40:52 [RudiNode info] keywords: ['2010', '2014', '2018', 'Agriculture', 'Aires de jeux', 'Autres filières', 'BP', 'Bioénergies', 'CA', 'Catalogue', 'Cogénération', 'Commune', 'Compte administratif', 'Consommation électrique', 'Culture', 'Diabète', 'Défibrillateurs', 'ENS', 'Economie', 'Election', 'Energie', 'Enfance', 'Entreprises', 'Eolien', 'Espace public', 'Etablissements', 'Hydraulique', 'IRIS', 'Industrie', 'Locale', 'Loisirs', 'Metadata', 'Metadonnées', 'Mobilité', 'NAF', 'PLU', 'Photovoltaïque', 'Population', 'Prénoms', 'Référentiels géographiques', 'Réseaux', 'SDES', 'SIREN', 'SIRET', 'Santé', 'Secteur d’activité', 'Sirene', 'Thermosensibilité', 'Thermosensible', 'Thorigné-Fouillard', 'Toilettes publiques', 'Usages', 'Usages non th

## Filtering the metadata
`RudiNodeReader` object offers some tools to filter the metadata with a partial JSON.
> You will need to understand how a RUDI metadata is structured to create adequate filters. See RUDI node [external API](https://app.swaggerhub.com/apis/OlivierMartineau/RUDI-PRODUCER) documentation for this.

> All the elements given in the filter are matched in the metadata that are kept in the end result of the filtering operation.

In [11]:
filter_tag = 'Filtering metadata'
example_filter = {'producer': {'organization_id': '1d6bc543-07ed-46f6-a813-958edb73d5f0', 'organization_name': 'SIB (Test)'}}
log_d(filter_tag, 'with JSON', rudi_node_info.filter_metadata(example_filter))

D 2023-04-18 17:40:54 [Filtering metadata] with JSON: [{'global_id': '053591c6-8922-46ac-9964-58ea41087663', 'resource_title': 'Statistiques_Questionnaire_Diabete', 'synopsis': [{'lang': 'fr', 'text': 'Statistiques d\'usage des questionnaires grand public dans le cadre de la responsabilité populationnelle de la pathologie "Diabète de type 2"'}], 'summary': [{'lang': 'fr', 'text': 'XX'}], 'theme': 'health', 'keywords': ['Diabète', 'Santé'], 'producer': {'organization_id': '1d6bc543-07ed-46f6-a813-958edb73d5f0', 'organization_name': 'SIB (Test)'}, 'contacts': [{'contact_id': '70e09aca-a4cf-4f75-92f7-0242a6194312', 'contact_name': 'Bacasable', 'email': 'bacasable@rudi.fr'}], 'available_formats': [{'checksum': {'algo': 'SHA-256', 'hash': '3211736d9f46ab98dd9cffe0ea3136a6e890f67f57e281b11b396b75e672df47'}, 'media_dates': {'created': '2023-03-22T15:09:34.614Z', 'updated': '2023-03-22T15:09:34.614Z'}, 'connector': {'url': 'https://bacasable.fenix.rudi-univ-rennes1.fr/media/download/bce8319b-d

Some shortcuts have been implemented to make it easier to filter the metadata:

In [12]:

meta_producer = 'Univ. Rennes'
log_d(filter_tag, f"with producer name '{meta_producer}'", rudi_node_info.get_metadata_with_producer(meta_producer))

meta_contact = 'Bacasable'
log_d(filter_tag, f"with contact name '{meta_contact}'", rudi_node_info.get_metadata_with_contact(meta_contact))

meta_theme = 'citizenship'
log_d(filter_tag, f"with theme '{meta_theme}'", rudi_node_info.get_metadata_with_theme(meta_theme))

meta_keywords = ['répartition', 'Commune']
log_d(filter_tag, f"with keywords '{meta_keywords}'", rudi_node_info.get_metadata_with_keywords(meta_keywords))

log_d(filter_tag, f"with available media", rudi_node_info.metadata_with_available_media)


D 2023-04-18 17:40:57 [Filtering metadata] with producer name 'Univ. Rennes': []
D 2023-04-18 17:40:57 [Filtering metadata] with contact name 'Bacasable': [{'global_id': 'f48b4bcd-bba3-47ba-86e6-c0754b748728', 'resource_title': 'parcours pédestre sur la ville de rennes', 'synopsis': [{'lang': 'fr', 'text': 'chemins pédestres de rennes '}], 'summary': [{'lang': 'fr', 'text': 'chemins pédestres de rennes '}], 'theme': 'publicSpace', 'keywords': ['bike', 'city', 'demo', 'municipality', 'population', 'transport', 'wind'], 'producer': {'organization_id': 'ae643f29-73f1-468b-ab90-e3430a1a2fa2', 'organization_name': 'RUDI', 'organization_address': 'RUDI'}, 'contacts': [{'contact_id': '70e09aca-a4cf-4f75-92f7-0242a6194312', 'contact_name': 'Bacasable', 'email': 'bacasable@rudi.fr'}], 'available_formats': [{'checksum': {'algo': 'MD5', 'hash': 'd739ab9bd2f3deacb97630227d4de400'}, 'media_dates': {'created': '2023-03-03T11:15:57.226Z', 'updated': '2023-03-03T11:15:57.226Z'}, 'connector': {'url': '

Additional code is provided to find a metadata:

In [13]:
find_tag = 'Finding a metadata'

meta_id = 'f48b4bcd-bba3-47ba-86e6-c0754b748728'
log_d(find_tag, f"with metadata uuid '{meta_id}'", rudi_node_info.find_metadata_with_uuid(meta_id))

meta_title = 'parcours pédestre sur la ville de rennes'
log_d(find_tag, f"with metadata title '{meta_title}'", rudi_node_info.find_metadata_with_title(meta_title))

file_name = 'toucan.jpg'
log_d(find_tag, f"with file name '{file_name}'", rudi_node_info.find_metadata_with_media_name(file_name))

file_uuid = '782bab2d-7ee8-4633-9c0a-173649b4d879'
log_d(find_tag, f"with file uuid '{file_uuid}'", rudi_node_info.find_metadata_with_media_uuid(file_uuid))


D 2023-04-18 17:40:59 [Finding a metadata] with metadata uuid 'f48b4bcd-bba3-47ba-86e6-c0754b748728': {'global_id': 'f48b4bcd-bba3-47ba-86e6-c0754b748728', 'resource_title': 'parcours pédestre sur la ville de rennes', 'synopsis': [{'lang': 'fr', 'text': 'chemins pédestres de rennes '}], 'summary': [{'lang': 'fr', 'text': 'chemins pédestres de rennes '}], 'theme': 'publicSpace', 'keywords': ['bike', 'city', 'demo', 'municipality', 'population', 'transport', 'wind'], 'producer': {'organization_id': 'ae643f29-73f1-468b-ab90-e3430a1a2fa2', 'organization_name': 'RUDI', 'organization_address': 'RUDI'}, 'contacts': [{'contact_id': '70e09aca-a4cf-4f75-92f7-0242a6194312', 'contact_name': 'Bacasable', 'email': 'bacasable@rudi.fr'}], 'available_formats': [{'checksum': {'algo': 'MD5', 'hash': 'd739ab9bd2f3deacb97630227d4de400'}, 'media_dates': {'created': '2023-03-03T11:15:57.226Z', 'updated': '2023-03-03T11:15:57.226Z'}, 'connector': {'url': 'https://bacasable.fenix.rudi-univ-rennes1.fr/media/dow

## Downloading a file
`RudiNodeReader` object also provides a method to download the data stored on the node:

In [14]:
dwnld_tag = 'Downloading'
dwnld_dir = './dwnld'

log_d(dwnld_tag, f"media with uuid '{file_uuid}'", rudi_node_info.download_file_with_uuid(file_uuid, dwnld_dir))
log_d(dwnld_tag, f"media with name '{file_name}'", rudi_node_info.download_file_with_name(file_name, dwnld_dir))
log_d(dwnld_tag, f"media for metadata '{meta_id}'", rudi_node_info.download_files_for_metadata(meta_id, dwnld_dir))


D 2023-04-18 17:41:01 [media_download] content saved to file: /Users/omartine/Dev/_Projets/Rudi/rudinode-read/dwnld/unicorn.png
D 2023-04-18 17:41:01 [Downloading] media with uuid '782bab2d-7ee8-4633-9c0a-173649b4d879': {'status': 'downloaded', 'media': {'media_name': 'unicorn.png', 'media_id': '782bab2d-7ee8-4633-9c0a-173649b4d879', 'media_url': 'https://bacasable.fenix.rudi-univ-rennes1.fr/media/download/782bab2d-7ee8-4633-9c0a-173649b4d879', 'file_type': 'image/png', 'created': '2022-01-21T10:40:28.781Z', 'updated': '2022-01-21T10:40:28.781Z', 'file_path': '/Users/omartine/Dev/_Projets/Rudi/rudinode-read/dwnld/unicorn.png'}}
D 2023-04-18 17:41:01 [media_download] content saved to file: /Users/omartine/Dev/_Projets/Rudi/rudinode-read/dwnld/toucan.jpg
D 2023-04-18 17:41:01 [Downloading] media with name 'toucan.jpg': {'status': 'downloaded', 'media': {'media_name': 'toucan.jpg', 'media_id': '854aa5ba-9c9a-474e-bf08-390b74d51b69', 'media_url': 'https://bacasable.fenix.rudi-univ-rennes1.

## Dumping the metadata into a file
`RudiNodeReader` object also provides a method to dump the metadata into a local file:

In [15]:
rudi_node_info.save_metadata_to_file(dwnld_dir, 'rudi_node_metadata.json')
